In [62]:
import torch
from torch import nn
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch.optim as optim

In [63]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [64]:
# Load data (one large matrix?)
datafiles = ['../project/dataset1.csv', '../project/dataset2.csv', '../project/dataset3.csv', '../project/dataset4.csv', '../project/dataset5.csv']
n_samples = 50000           # Number of samples
n_dataset = len(datafiles)  # Number of datasets
train = 0.9   # 245000 (77.777...%)
test = 0.1    # 35000 (10%)
# val = 1-train # 70000 (Rest)

size_test = int(n_samples*n_dataset*test)+1
size_train = int((n_samples*n_dataset-size_test)*train)+1
# size_val = int((n_samples*n_dataset-size_test)*val)+1

In [65]:
# Split all datasets
datasets = np.empty((int(n_samples*n_dataset),5))
print(datasets.shape)
for i,datafile in enumerate(datafiles):
  dataset = np.loadtxt(datafile, delimiter=',')
  datasets[int(i*n_samples):int(n_samples*(i+1)),:] = dataset
print(datasets[0,0])
# Split into input and output
Cr = datasets[:,0:4]   # Input: relative entropy of coherence
ES = np.array([float(y) for y in datasets[:,4]])  # Output: entangled/separable

# Count number of entangled and separable states
n_entangled = len([es for es in ES if es == 1])
print(f'Number of entangled states: {n_entangled}\n')
print(f'Number of separable states: {n_samples*n_dataset-n_entangled}\n')
# Split into train-and-validation and test
#break datasets into train and test

Cr_train, Cr_test, ES_train, ES_test = train_test_split(Cr, ES, test_size=test,
                                                      random_state=42)
# # Split into train and validation
# Cr_train, Cr_val, ES_train, ES_val = train_test_split(Cr_trainval, ES_trainval, test_size=val,
#                                                       random_state=30)
# DEBUG
print(np.shape(Cr_train))
print(np.shape(Cr_test))
print(np.shape(ES_train))
print(np.shape(ES_test))

(250000, 5)
0.195132062642601
Number of entangled states: 127330

Number of separable states: 122670

(225000, 4)
(25000, 4)
(225000,)
(25000,)


In [66]:
# Tansform data into tensors
Cr_tensor = torch.tensor(Cr, dtype=torch.float32)
ES_tensor = torch.tensor(ES, dtype=torch.float32)
# Cr_train = torch.tensor(Cr_train, dtype=torch.float32)
# Cr_test = torch.tensor(Cr_test, dtype=torch.float32)
# ES_train = torch.tensor(ES_train, dtype=torch.float32)
# ES_test = torch.tensor(ES_test, dtype=torch.float32)

# # In case we have a gpu
Cr_tensor = Cr_tensor.to(device)
ES_tensor = ES_tensor.to(device)
# Cr_train = Cr_train.to(device)
# Cr_test = Cr_test.to(device)
# ES_train = ES_train.to(device)
# ES_test = ES_test.to(device)

In [67]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader
tensor_dataset = TensorDataset(Cr_tensor, ES_tensor)
dataset = DataLoader(tensor_dataset, batch_size=32, shuffle=True)

In [68]:
# Define the neural network architecture
class NeuralNetwork_3layer_with_a0(nn.Module):
    def __init__(self, n_nodes):
        super().__init__()
        self.layer1 = nn.Linear(4, n_nodes)
        self.layer2 = nn.Linear(n_nodes,5)
        self.layer3 = nn.Linear(5, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.apply(self._init_weights)


    def forward(self, x):
      # if input x provided, describe how to produce output tensor
        x = self.relu(self.layer1(x))
        x = self.sig(self.layer2(x))
        x = self.sig(self.layer3(x))
        return x

    def _init_weights(self, l):
      # Initialize weights uniformly
      if isinstance(l, nn.Linear):
        nn.init.kaiming_uniform_(l.weight)
        #nn.init.uniform_(l.weight)
        l.bias.data.fill_(0.01)

In [77]:
def train_model(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Train the model
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        y = y.view(-1, 1)
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_model(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            # Reshape y to match pred's shape
            y = y.view(-1, 1)  # Assuming y has shape (batch_size, 1)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [78]:
model = NeuralNetwork_3layer_with_a0(50).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_model(dataset, model, loss_fn, optimizer)
    test_model(dataset, model, loss_fn)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.308821 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.288281 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.269343 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.232246 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.196962 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 93.0%, Avg loss: 0.170735 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 93.9%, Avg loss: 0.153375 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 94.2%, Avg loss: 0.143128 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 94.4%, Avg loss: 0.139010 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.128896 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 94.6%, Avg los

KeyboardInterrupt: 

In [71]:
# Train model without inequality
model = LinearRegressionWithHidden   # Define model you want to train
epochs = 50        # Number of epochs
batches = 32       # Batch size

start = 100      # Limit for early stopping
stop_th = 50     # Early stopping threshold

best_acc = -1   # Save best accuracy
best_e = -1     # Save best epoch
losses = []     # Save epoch losses
accs_val = []  # Save validation accuracy
accs_train = [] # Save train accuracy
count = 1       # Count for lower limit for early stopping

# Define Loss function - binary cross-entropy
loss_fn = nn.BCELoss()

# Define optimizer - RMSprop optimizer with default hyperparameters
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)#,alpha=0.99,eps=1e-7)

# Definition to extract P_i-values
model.layer2.register_forward_hook(get_activation('layer2'))

for e in range(epochs):
    # Training the model on training data set
    model.train()   # Model put into training mode
    for b in range(0, len(Cr_train), batches):
      ES_pred = model(Cr_train[b:b+batches,:])   # Train
      loss = loss_fn(ES_pred, ES_train[b:b+batches,:]) # Calculate loss
      optimizer.zero_grad()   # Resets gradients
      loss.backward()         # Calculate gradients
      optimizer.step()        # Update weights

    # Evaluation on training and validation data set
    model.eval()            # Evaluation
    ES_pred_val = model(Cr_test)       # Evaluation validation data
    alphas_val = torch.sigmoid(activation['layer2'])
    ES_pred_train = model(Cr_train_norm)   # Evaluation train data
    alphas_train = torch.sigmoid(activation['layer2'])
    #print(alphas_val)
    #print(alphas_train)

    # Evaluating inequality
    ES_ineq_train = ineq(alphas_train, Cr_train)
    ES_ineq_val = ineq(alphas_val, Cr_val)
    acc_ineq_train = (ES_ineq_train == ES_train).float().mean()
    acc_ineq_val = (ES_ineq_val == ES_val).float().mean()
    TS, FE, FS, TE = confusion_matrix(ES_train.cpu().detach().numpy(), ES_ineq_train.cpu().detach().numpy()).ravel()
    print(f'For epoch {e}, inequality accuracy = {acc_ineq_train*100}%\n')
    print(f'TS: {TS} FE: {FE} FS: {FS} TE: {TE}\n')
    #print(f'Alphas: {alphas[1:10]}\n')

    # Calculate and save loss
    loss = loss_fn(ES_pred_train, ES_train)
    losses.append(loss.cpu().detach().numpy())

    # Calculating and saving accuracies
    acc_val = (ES_pred_val.round() == ES_val).float().mean()   # Accuracy validation set
    acc_train = (ES_pred_train.round() == ES_train).float().mean()   # Accuracy train set
    accs_val.append(acc_val.cpu().detach().numpy())   # For plotting
    accs_train.append(acc_train.cpu().detach().numpy())   # For plotting
    print(f'For epoch {e}, validation accuracy = {acc_val*100}%\n')

    # Printing!
    print(f'Epoch {e}!\n')
    print(f'Loss: {loss}')


    # Checkpointing and early stopping
    if acc_val > best_acc:
        best_acc = acc_val
        best_e = e
        checkpoint(model, 'best_model.pth')
    elif (count > start) and (e - best_e > stop_th):
        print(f'Early stop at epoch = {e}\n')
        break
    count += 1

resume(model, 'best_model.pth')   # Use best model

TypeError: Module.parameters() missing 1 required positional argument: 'self'

In [ ]:
Cr_trainval_tensor.shape, ES_trainval_tensor.shape, Cr_trainval.shape, type(Cr_trainval)

(torch.Size([200000, 4]), torch.Size([200000]), (200000, 4), numpy.ndarray)

In [ ]:
from torch.utils.data import Dataset, DataLoader
model = LinearRegressionWithHidden()

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Split data into train and test sets
Cr_trainval, Cr_test, ES_trainval, ES_test = train_test_split(Cr, ES, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
# Cr = X Es = Y
Cr_trainval_tensor = torch.tensor(Cr_trainval, dtype=torch.float32)
ES_trainval_tensor = torch.tensor(ES_trainval, dtype=torch.float32)
#merge the data Cr_trainval_tensor ES_trainval_tensor
Cr_trainval = torch.utils.data.TensorDataset(Cr_trainval_tensor, ES_trainval_tensor)
Cr_test = torch.utils.data.TensorDataset(Cr_test,ES_test)
train_loader = DataLoader(Cr_trainval, batch_size=10, shuffle=True)
test_loader = DataLoader(Cr_test, batch_size=10, shuffle=True)


# Training loop


TypeError: 'int' object is not callable

In [ ]:
for x in train_loader:
    print(x.shape)
    print(x)
    break

torch.Size([10, 4])
tensor([[2.8292e-01, 5.0141e-01, 2.8292e-01, 1.0699e-03],
        [6.0408e-01, 6.5331e-01, 6.3826e-01, 5.8904e-01],
        [7.4930e-02, 3.8622e-02, 4.4818e-02, 7.8443e-02],
        [7.4905e-02, 6.5564e-02, 3.4092e-02, 6.6149e-02],
        [9.3140e-02, 1.4367e-01, 9.3140e-02, 7.2529e-03],
        [2.3706e-01, 2.3727e-01, 2.3706e-01, 1.3707e-01],
        [1.1822e+00, 1.0417e+00, 1.1822e+00, 5.9894e-01],
        [4.1536e-01, 5.8036e-01, 3.6490e-01, 1.9990e-01],
        [6.6071e-02, 6.0675e-02, 6.6071e-02, 4.0399e-02],
        [3.3015e-01, 3.4334e-01, 3.5657e-01, 3.4338e-01]], dtype=torch.float64)


In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(Cr_trainval_tensor)
    ES_trainval_tensor = ES_trainval_tensor.view(-1, 1)

    # Calculate the loss
    loss = criterion(outputs, ES_trainval_tensor)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, you can evaluate the model on the test set
with torch.no_grad():
    Cr_test_tensor = torch.tensor(Cr_test, dtype=torch.float32)
    ES_test_tensor = torch.tensor(ES_test, dtype=torch.float32)

    # Forward pass through the model to get predictions
    outputs = model(Cr_test_tensor)

    # Convert the output tensor to have the same shape as the target tensor
    outputs = outputs.view(-1)  # Ensure the output tensor is 1D

    # Calculate the test loss
    test_loss = criterion(outputs, ES_test_tensor)

    print(f'Test Loss: {test_loss.item():.4f}')